In [1]:
import pandas as pd
import numpy as np

file_to_load = "purchase_data.csv"

purchase_data = pd.read_csv(file_to_load)

In [15]:
total_player = len(purchase_data["SN"].unique())            #用unique找出沒有重複的人
df1 = pd.DataFrame({"Total Players":[total_player]})
df1

,Total Players
0,576


In [30]:
unique_items = len(purchase_data["Item ID"].unique())   #用unique找出沒有重複的items

a = round(purchase_data["Price"].mean(), 2)    #取平均值 小數點第二位
avg_price = f"${a}"

num_pur = len(purchase_data["Purchase ID"])   #購買的數量

b = purchase_data["Price"].sum()             #價格總和
total_rev = f"${b}"

df2 = pd.DataFrame({"Number of Unique Items":[unique_items], 
                    "Average Price":[avg_price], 
                    "Number of Purchases":[num_pur], 
                    "Total Revenue": [total_rev]})
df2

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [78]:
men_num = len(purchase_data.loc[purchase_data["Gender"] == "Male", "SN"].unique())     #找出男生(女生) 且名字不重複
women_num = len(purchase_data.loc[purchase_data["Gender"] == "Female", "SN"].unique())
other_num = total_player - (men_num + women_num)    #其餘的用 總和扣掉 男+女

men_per = round((men_num / total_player)*100, 2)
women_per = round((women_num / total_player)*100, 2)
other_per = round((other_num / total_player)*100, 2)  #比率就是 人數/總人數

df3 = pd.DataFrame({"Percentage of Players":[men_per, women_per, other_per], 
                    "Total Count":[men_num, women_num, other_num]},
                   index = ["Male", "Female", "Other / Non-Disclosed"])
df3

,Percentage of Players,Total Count
Male,84.03,484
Female,14.06,81
Other / Non-Disclosed,1.91,11


In [120]:
all_pur_count = len(purchase_data["Gender"]) #780

#purchase count
men_pur_count = (purchase_data["Gender"] == "Male").sum() #652
women_pur_count = (purchase_data["Gender"] == "Female").sum() #113
other_pur_count = all_pur_count - (men_pur_count + women_pur_count) #15

#total purchase value
all_pur_money = purchase_data["Price"].sum()
men_pur_money = purchase_data.loc[purchase_data["Gender"] == "Male", "Price"].sum()
women_pur_money = purchase_data.loc[purchase_data["Gender"] == "Female", "Price"].sum()
other_pur_money = round((all_pur_money - (men_pur_money + women_pur_money)), 2)

#average purchase total per person
men_avg_pur =  f"${round((men_pur_money/men_num), 2)}"
women_avg_pur = f"${round((women_pur_money/women_num), 2)}"
other_avg_pur = f"${round((other_pur_money/other_num), 2)}"

#average purchase price
men_price_avg = round((men_pur_money/men_pur_count),5)
women_price_avg = round((women_pur_money/women_pur_count), 5)
other_price_avg = round((other_pur_money/other_pur_count), 3)

df4 = pd.DataFrame({"Purchase Count":[women_pur_count, men_pur_count, other_pur_count],
                   "Average Purchase Price":[women_price_avg, men_price_avg, other_price_avg],
                   "Total Purchase Value":[women_pur_money, men_pur_money, other_pur_money],
                   "Avg Purchase Total Per Person":[women_avg_pur, men_avg_pur, other_avg_pur]},
                   index = ["Female", "Male", "Other / Non-Disclosed"])

df4.index.name = "Gender"
df4

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Purchase Total Per Person
Gender,,,,
Female,113,3.20301,361.94,$4.47
Male,652,3.01785,1967.64,$4.07
Other / Non-Disclosed,15,3.34600,50.19,$4.56


In [248]:
bins = [0,9,14,19,24,29,34,39,199]
label = ["<10","10-14","15-19","20-24","25-29", "30-34","35-39","40+"]
player = purchase_data.loc[:,["SN","Age"]].groupby("SN").mean()  #先取出"SN","Age" 再利用SN來分類分群

age_layer = pd.cut(player["Age"], bins, labels = label) # 進行表格製作 設定以label bin為分差的圖

df5 = pd.DataFrame(age_layer.value_counts().reindex(label)) #照label的順序重新排
df5["Percentage of Players"] = round(((df5["Age"] / total_player)*100),2) #計算所占比例
df5.rename(columns = {'Age':'Total Count'}, inplace = True)

df5 = df5.reindex(columns=sorted(df5.columns))
df5

,Percentage of Players,Total Count
<10,2.95,17
10-14,3.82,22
15-19,18.58,107
20-24,44.79,258
25-29,13.37,77
30-34,9.03,52
35-39,5.38,31
40+,2.08,12


In [261]:
bins = [0, 9, 14, 19, 24, 29, 34, 39, 190]
label = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#apply the dataframe in the new form in terms of label form
new_data = purchase_data.copy()

# transform Age column into our bins' way
new_data["Age"] = pd.cut(new_data["Age"], bins, labels = label)

# get average purchase price of each bins
avg_pur_price = []
for i in label:
    each = new_data.loc[new_data["Age"] == i]["Price"].mean()
    each = "%.2f" % each
    each = f"${each}"
    avg_pur_price.append(each)

    
#get total purchase values for each bins
total_pur_value = []
groupby_form = (new_data.groupby("Age")["Price"].sum())
for i in range(len(groupby_form)):
    a = "%.2f" % groupby_form[i]
    a = f"${a}"
    total_pur_value.append(a)
    
#get average total per person for each bins
avg_total_per_person = []
form = new_data.groupby("Age")["Price"].sum()
for i in range(len(form)):
    a = form[i] / df5["Total Count"][i]
    a = "%.2f" % a
    a = f"${a}"
    avg_total_per_person.append(a)
    
new_data = pd.DataFrame(new_data.groupby("Age")["Purchase ID"].count())
new_data.rename(columns = {'Purchase ID':'Purchase Count'}, inplace = True)
new_data["Average Purchase Price"] = avg_pur_price
new_data["Total Purchase Value"] = total_pur_value
new_data["Average Purchase Total per Person"] = avg_total_per_person

df6 = new_data
df6

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Age,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [415]:
name = pd.DataFrame(purchase_data.groupby("SN")["Price"].count())  #用SN分群 再以Price計算count出來的多寡
name.columns = ["Purchase Count"]

avg_pur_price = pd.DataFrame(purchase_data.groupby("SN")["Price"].mean().map("${:.2f}".format)) #取平均
name["Average Purchase Price"] = avg_pur_price

total_pur_price = pd.DataFrame(purchase_data.groupby("SN")["Price"].sum()) #取總額
name["Total Purchase Value"] = total_pur_price

name = name.sort_values(by = "Total Purchase Value",ascending = 0) #重新排列 高至低
df7 = name.head()


df7["Total Purchase Value"] = df7["Total Purchase Value"].map("${:.2f}".format)
df7

<ipython-input-415-ca899a27d879>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7["Total Purchase Value"] = df7["Total Purchase Value"].map("${:.2f}".format)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [393]:
small_data = purchase_data[["Item ID", "Item Name", "Price"]] #Retrieve the Item ID, Item Name, and Item Price columns
group = small_data.groupby(["Item ID", "Item Name"])    #Group by Item ID and Item Name


small_data = pd.DataFrame(group.mean())
small_data.rename(columns = {"Price":"Item Price"}, inplace = True) #更改column的名字

pur_Count = group.count() #計算第二行的Groupby數量
small_data["Purchase Count"] = pur_Count

total_pur_value = group.sum()  #計算第二行的Groupby總數
small_data["Total Purchase Value"] = total_pur_value


small_data = small_data.sort_values(by = "Purchase Count", ascending = False).head()
small_data = small_data.reindex(columns = ["Purchase Count", "Item Price", "Total Purchase Value"])
small_data["Item Price"]

#下面的只是單純為每個small data的column值 多加一個$的符號
kkk = []
for a in (small_data["Item Price"]):
    a = "%.2f" % a
    a = f"${a}"
    kkk.append(a)
small_data.loc[:,"Item Price"] = kkk

mmm = []
for b in (small_data["Total Purchase Value"]):
    b = "%.2f" % b
    b = f"${b}"
    mmm.append(b)

small_data.loc[:,"Total Purchase Value"] = mmm
df8 = small_data

df8

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [409]:
#一樣是找到count, sum, mean 接著再進行欄位的位置調整
#還有名字更改成Item Price
profit_data = purchase_data[["Item ID", "Item Name", "Price"]]
group_by = profit_data.groupby(["Item ID", "Item Name"])

profit_data = pd.DataFrame(group_by.mean())
profit_data.rename(columns = {"Price":"Item Price"}, inplace = True)

purchase_count = group_by.count()
profit_data["Purchase Count"] = purchase_count

profit_total = group_by.sum()
profit_data["Total Purchase Value"] = profit_total
profit_data = profit_data.sort_values(by = "Total Purchase Value", ascending = False).head()

profit_data = profit_data.reindex(columns = ["Purchase Count", "Item Price", "Total Purchase Value"])

#下面的只是單純為每個small dara的column值 多加一個$的符號
kkk = []
for a in (profit_data["Item Price"]):
    a = "%.2f" % a
    a = f"${a}"
    kkk.append(a)
small_data.loc[:,"Item Price"] = kkk

mmm = []
for b in (profit_data["Total Purchase Value"]):
    b = "%.2f" % b
    b = f"${b}"
    mmm.append(b)

profit_data.loc[:,"Total Purchase Value"] = mmm
df9 = profit_data

df9

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,$50.76
82,Nirvana,9,4.90,$44.10
145,Fiery Glass Crusader,9,4.58,$41.22
92,Final Critic,8,4.88,$39.04
103,Singed Scalpel,8,4.35,$34.80
